In [ ]:
!git clone https://github.com/state-spaces/mamba.git

Cloning into 'mamba'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 304 (delta 123), reused 98 (delta 97), pack-reused 137
Receiving objects: 100% (304/304), 508.35 KiB | 4.89 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [ ]:
# %%capture
# # Installing SpeechBrain via pip
# BRANCH = 'develop'
# !python -m pip install git+https://github.com/speechbrain/speechbrain.git@$BRANCH

# Clone SpeechBrain repository
%%capture
# Local installation
!git clone https://github.com/speechbrain/speechbrain/
%cd /content/speechbrain/
!pip install -r requirements.txt
!pip install -e .

In [ ]:
!wget https://www.openslr.org/resources/12/train-clean-100.tar.gz
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz
!wget https://www.openslr.org/resources/12/test-clean.tar.gz

--2024-04-14 00:42:40--  https://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://openslr.elda.org/resources/12/train-clean-100.tar.gz [following]
--2024-04-14 00:42:40--  https://openslr.elda.org/resources/12/train-clean-100.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘train-clean-100.tar.gz’

train-clean-100.tar 100%[===================>]   5.95G  27.1MB/s    in 3m 43s  

2024-04-14 00:46:24 (27.3 MB/s) - ‘train-clean-100.tar.gz’ saved [6387309499/6387309499]

--2024-04-14 00:46:24--  https://www.openslr.org/resources/12/dev-clean.tar.gz
Res

In [ ]:
!tar -zxf dev-clean.tar.gz
!tar -zxf test-clean.tar.gz
!tar -zxf train-clean-100.tar.gz

In [ ]:
!pip install causal-conv1d>=1.2.0

In [ ]:
!pip install mamba-ssm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.4 MB/s eta 0:00:00
  Created wheel for mamba-ssm: filename=mamba_ssm-1.2.0.post1-cp310-cp310-linux_x86_64.whl size=137750683 sha256=b264292652a34fb9dd0ce880a34a4407ba7256a3338388d056769ec29a4581c9
  Stored in directory: /root/.cache/pip/wheels/22/6e/60/ddd5c574b5793a30028f2cabdacd2a3ec2276edaaa8c00fd35
Successfully built mamba-ssm


In [ ]:
!pip install unidecode tgt
!pip install git+https://github.com/huggingface/transformers@main



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers (to revision main) to /tmp/pip-req-build-q11qwzms
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-q11qwzms
  Resolved https://github.com/huggingface/transformers to commit b109257f4fb8b1166e7c53cc5418632014ed53a5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8888292 sha256=27664645591b54a441389dd3726e53d66da5571551ae4f478caf9cb452c208f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ql6o_wbb/wheels/d9/3d/ab/28ae056a634730dae1213fc3321afc3fc1d207699fe3f889cf
Successfully built transformers
  Attempting uninstall: transformers
 

In [ ]:
%cd /content/speechbrain/speechbrain/lobes/models

/content/speechbrain/speechbrain/lobes/models


In [ ]:

%%file mamba.py

import torch

import speechbrain as sb

from transformers import MambaConfig, MambaModel
import torch
import torch as nn



class MultiHeadAttentionBlock(torch.nn.Module):
  def __init__(self, d_model: int, num_heads: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.dropout = torch.nn.Dropout(dropout)
        self.head_dim = d_model // num_heads

        self.w_q = torch.nn.Linear(d_model, d_model)
        self.w_k = torch.nn.Linear(d_model, d_model)
        self.w_v = torch.nn.Linear(d_model, d_model)
        self.w_o = torch.nn.Linear(d_model, d_model)

  @staticmethod
  def attention(q, k, v, mask, dropout: torch.nn.Dropout):
      head_dim = q.shape[-1]

      # (batch_size, num_heads, seq_len, head_dim) * (batch_size, num_heads, head_dim, seq_len) = (batch_size, num_heads, seq_len, seq_len
      attention_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(head_dim, dtype=torch.float32))

      if mask is not None:
          attention_scores = attention_scores.masked_fill(mask == 0, -1e9)

      attention_scores = torch.softmax(attention_scores, dim=-1) # (batch_size, num_heads, seq_len, seq_len)

      if dropout is not None:
          attention_scores = dropout(attention_scores)

      return torch.matmul(attention_scores, v), attention_scores


  def forward(self, q, k, v, mask):
        querry = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        # Split the querry, key and value into num_heads
        # Shape: (batch_size, seq_len, num_heads, head_dim)
        querry = querry.view(querry.shape[0], -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        key = key.view(key.shape[0], -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        value = value.view(value.shape[0], -1, self.num_heads, self.head_dim).permute(0, 2, 1, 3) # (batch_size, num_heads, seq_len, head_dim)
        x, self.attention_scores = self.attention(querry, key, value, mask, self.dropout)

        # Concatenate the heads
        # Shape: (batch_size, seq_len, d_model)
        x = x.permute(0, 2, 1, 3).contiguous().view(x.shape[0], -1, self.d_model)

        return self.w_o(x)


class MambaEncoder(torch.nn.Module):
    def __init__(
        self,
        vocab_size,
        hidden_size,
        state_size,
        num_hidden_layers,
        layer_norm_epsilon,
        pad_token_id,
        bos_token_id,
        eos_token_id,
        expand,
        conv_kernel,
        use_bias,
        use_conv_bias,
        hidden_act,
        initializer_range,
        residual_in_fp32,
        time_step_rank,
        time_step_scale,
        time_step_min,
        time_step_max,
        time_step_init_scheme,
        time_step_floor,
        rescale_prenorm_residual,
        use_cache
      ):
            super().__init__()
            self.config = MambaConfig(
            vocab_size=vocab_size,
            hidden_size=hidden_size,
            state_size=state_size,
            num_hidden_layers=num_hidden_layers,
            layer_norm_epsilon=layer_norm_epsilon,
            pad_token_id=pad_token_id,
            bos_token_id=bos_token_id,
            eos_token_id=eos_token_id,
            expand=expand,
            conv_kernel=conv_kernel,
            use_bias=use_bias,
            use_conv_bias=use_conv_bias,
            hidden_act=hidden_act,
            initializer_range=initializer_range,
            residual_in_fp32=residual_in_fp32,
            time_step_rank=time_step_rank,
            time_step_scale=time_step_scale,
            time_step_min=time_step_min,
            time_step_max=time_step_max,
            time_step_init_scheme=time_step_init_scheme,
            time_step_floor=time_step_floor,
            rescale_prenorm_residual=rescale_prenorm_residual,
            use_cache=use_cache
        )
            self.mamba = MambaModel(self.config)
    def forward(self,inputs_embeds = None):
            enc_out = self.mamba(inputs_embeds = inputs_embeds)
            return enc_out.last_hidden_state


class MambaDecoder(torch.nn.Module):
    def __init__(
        self,
        vocab_size,
        hidden_size,
        state_size,
        num_hidden_layers,
        layer_norm_epsilon,
        pad_token_id,
        bos_token_id,
        eos_token_id,
        expand,
        conv_kernel,
        use_bias,
        use_conv_bias,
        hidden_act,
        initializer_range,
        residual_in_fp32,
        time_step_rank,
        time_step_scale,
        time_step_min,
        time_step_max,
        time_step_init_scheme,
        time_step_floor,
        rescale_prenorm_residual,
        use_cache
      ):
        super().__init__()
        self.config = MambaConfig(
              vocab_size=vocab_size,
              hidden_size=hidden_size,
              state_size=state_size,
              num_hidden_layers=num_hidden_layers,
              layer_norm_epsilon=layer_norm_epsilon,
              pad_token_id=pad_token_id,
              bos_token_id=bos_token_id,
              eos_token_id=eos_token_id,
              expand=expand,
              conv_kernel=conv_kernel,
              use_bias=use_bias,
              use_conv_bias=use_conv_bias,
              hidden_act=hidden_act,
              initializer_range=initializer_range,
              residual_in_fp32=residual_in_fp32,
              time_step_rank=time_step_rank,
              time_step_scale=time_step_scale,
              time_step_min=time_step_min,
              time_step_max=time_step_max,
              time_step_init_scheme=time_step_init_scheme,
              time_step_floor=time_step_floor,
              rescale_prenorm_residual=rescale_prenorm_residual,
              use_cache=use_cache
        )
        self.mamba = MambaModel(self.config)
        self.embedding = torch.nn.Embedding(self.config.vocab_size, self.config.hidden_size)
        self.cross_attention = MultiHeadAttentionBlock(self.config.hidden_size, 4)
        self.norm = torch.nn.LayerNorm(self.config.hidden_size)
        #self.fc = torch.nn.Linear(self.config.hidden_size, self.config.vocab_size)

    def forward(self, input_ids=None, embeds=None, encoder_output=None, src_mask=None, tgt_mask=None, cache_params=None):
        decoder_output = self.mamba(inputs_embeds=embeds, cache_params=cache_params)
        decoder_output = self.cross_attention(decoder_output.last_hidden_state, encoder_output, encoder_output, src_mask)
        decoder_output = self.norm(decoder_output)
        #logits = self.fc(decoder_output)

        return decoder_output

Overwriting mamba.py


In [ ]:
%cd /content/speechbrain/recipes/LibriSpeech/ASR
!mkdir mamba
%cd mamba




/content/speechbrain/recipes/LibriSpeech/ASR
/content/speechbrain/recipes/LibriSpeech/ASR/mamba


In [ ]:
%%file librispeech_prepare.py
"""
Data preparation.

Download: http://www.openslr.org/12

Author
------
 * Mirco Ravanelli, 2020
 * Ju-Chieh Chou, 2020
 * Loren Lugosch, 2020
 * Pierre Champion, 2023
 * Adel Moumen, 2024
"""

import csv
import functools
import logging
import os
import random
from collections import Counter
from dataclasses import dataclass

from speechbrain.dataio.dataio import (
    load_pkl,
    merge_csvs,
    read_audio_info,
    save_pkl,
)
from speechbrain.utils.data_utils import download_file, get_all_files
from speechbrain.utils.parallel import parallel_map

logger = logging.getLogger(__name__)
OPT_FILE = "opt_librispeech_prepare.pkl"
SAMPLERATE = 16000
OPEN_SLR_11_LINK = "http://www.openslr.org/resources/11/"
OPEN_SLR_11_NGRAM_MODELs = [
    "3-gram.arpa.gz",
    "3-gram.pruned.1e-7.arpa.gz",
    "3-gram.pruned.3e-7.arpa.gz",
    "4-gram.arpa.gz",
]


def prepare_librispeech(
    data_folder,
    save_folder,
    tr_splits=[],
    dev_splits=[],
    te_splits=[],
    select_n_sentences=None,
    merge_lst=[],
    merge_name=None,
    create_lexicon=False,
    skip_prep=False,
):
    """
    This class prepares the csv files for the LibriSpeech dataset.
    Download link: http://www.openslr.org/12

    Arguments
    ---------
    data_folder : str
        Path to the folder where the original LibriSpeech dataset is stored.
    save_folder : str
        The directory where to store the csv files.
    tr_splits : list
        List of train splits to prepare from ['test-others','train-clean-100',
        'train-clean-360','train-other-500'].
    dev_splits : list
        List of dev splits to prepare from ['dev-clean','dev-others'].
    te_splits : list
        List of test splits to prepare from ['test-clean','test-others'].
    select_n_sentences : int
        Default : None
        If not None, only pick this many sentences.
    merge_lst : list
        List of librispeech splits (e.g, train-clean, train-clean-360,..) to
        merge in a single csv file.
    merge_name: str
        Name of the merged csv file.
    create_lexicon: bool
        If True, it outputs csv files containing mapping between grapheme
        to phonemes. Use it for training a G2P system.
    skip_prep: bool
        If True, data preparation is skipped.

    Returns
    -------
    None

    Example
    -------
    >>> data_folder = 'datasets/LibriSpeech'
    >>> tr_splits = ['train-clean-100']
    >>> dev_splits = ['dev-clean']
    >>> te_splits = ['test-clean']
    >>> save_folder = 'librispeech_prepared'
    >>> prepare_librispeech(data_folder, save_folder, tr_splits, dev_splits, te_splits)
    """

    if skip_prep:
        return
    data_folder = data_folder
    splits = tr_splits + dev_splits + te_splits
    save_folder = save_folder
    select_n_sentences = select_n_sentences
    conf = {
        "select_n_sentences": select_n_sentences,
    }

    # Other variables
    # Saving folder
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    save_opt = os.path.join(save_folder, OPT_FILE)

    # Check if this phase is already done (if so, skip it)
    if skip(splits, save_folder, conf):
        logger.info("Skipping preparation, completed in previous run.")
        return
    else:
        logger.info("Data_preparation...")

    # Additional checks to make sure the data folder contains Librispeech
    check_librispeech_folders(data_folder, splits)

    # create csv files for each split
    all_texts = {}
    for split_index in range(len(splits)):
        split = splits[split_index]

        wav_lst = get_all_files(
            os.path.join(data_folder, split), match_and=[".flac"]
        )

        text_lst = get_all_files(
            os.path.join(data_folder, split), match_and=["trans.txt"]
        )

        text_dict = text_to_dict(text_lst)
        all_texts.update(text_dict)

        if select_n_sentences is not None:
            n_sentences = select_n_sentences[split_index]
        else:
            n_sentences = len(wav_lst)

        create_csv(save_folder, wav_lst, text_dict, split, n_sentences)

    # Merging csv file if needed
    if merge_lst and merge_name is not None:
        merge_files = [split_libri + ".csv" for split_libri in merge_lst]
        merge_csvs(
            data_folder=save_folder, csv_lst=merge_files, merged_csv=merge_name
        )

    # Create lexicon.csv and oov.csv
    if create_lexicon:
        create_lexicon_and_oov_csv(all_texts, save_folder)

    # saving options
    save_pkl(conf, save_opt)


def create_lexicon_and_oov_csv(all_texts, save_folder):
    """
    Creates lexicon csv files useful for training and testing a
    grapheme-to-phoneme (G2P) model.

    Arguments
    ---------
    all_texts : dict
        Dictionary containing text from the librispeech transcriptions
    save_folder : str
        The directory where to store the csv files.
    """
    # If the lexicon file does not exist, download it
    lexicon_url = "http://www.openslr.org/resources/11/librispeech-lexicon.txt"
    lexicon_path = os.path.join(save_folder, "librispeech-lexicon.txt")

    if not os.path.isfile(lexicon_path):
        logger.info(
            "Lexicon file not found. Downloading from %s." % lexicon_url
        )
        download_file(lexicon_url, lexicon_path)

    # Get list of all words in the transcripts
    transcript_words = Counter()
    for key in all_texts:
        transcript_words.update(all_texts[key].split("_"))

    # Get list of all words in the lexicon
    lexicon_words = []
    lexicon_pronunciations = []
    with open(lexicon_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            word = line.split()[0]
            pronunciation = line.split()[1:]
            lexicon_words.append(word)
            lexicon_pronunciations.append(pronunciation)

    # Create lexicon.csv
    header = "ID,duration,char,phn\n"
    lexicon_csv_path = os.path.join(save_folder, "lexicon.csv")
    with open(lexicon_csv_path, "w") as f:
        f.write(header)
        for idx in range(len(lexicon_words)):
            separated_graphemes = [c for c in lexicon_words[idx]]
            duration = len(separated_graphemes)
            graphemes = " ".join(separated_graphemes)
            pronunciation_no_numbers = [
                p.strip("0123456789") for p in lexicon_pronunciations[idx]
            ]
            phonemes = " ".join(pronunciation_no_numbers)
            line = (
                ",".join([str(idx), str(duration), graphemes, phonemes]) + "\n"
            )
            f.write(line)
    logger.info("Lexicon written to %s." % lexicon_csv_path)

    # Split lexicon.csv in train, validation, and test splits
    split_lexicon(save_folder, [98, 1, 1])


def split_lexicon(data_folder, split_ratio):
    """
    Splits the lexicon.csv file into train, validation, and test csv files

    Arguments
    ---------
    data_folder : str
        Path to the folder containing the lexicon.csv file to split.
    split_ratio : list
        List containing the training, validation, and test split ratio. Set it
        to [80, 10, 10] for having 80% of material for training, 10% for valid,
        and 10 for test.
    """
    # Reading lexicon.csv
    lexicon_csv_path = os.path.join(data_folder, "lexicon.csv")
    with open(lexicon_csv_path, "r") as f:
        lexicon_lines = f.readlines()
    # Remove header
    lexicon_lines = lexicon_lines[1:]

    # Shuffle entries
    random.shuffle(lexicon_lines)

    # Selecting lines
    header = "ID,duration,char,phn\n"

    tr_snts = int(0.01 * split_ratio[0] * len(lexicon_lines))
    train_lines = [header] + lexicon_lines[0:tr_snts]
    valid_snts = int(0.01 * split_ratio[1] * len(lexicon_lines))
    valid_lines = [header] + lexicon_lines[tr_snts : tr_snts + valid_snts]
    test_lines = [header] + lexicon_lines[tr_snts + valid_snts :]

    # Saving files
    with open(os.path.join(data_folder, "lexicon_tr.csv"), "w") as f:
        f.writelines(train_lines)
    with open(os.path.join(data_folder, "lexicon_dev.csv"), "w") as f:
        f.writelines(valid_lines)
    with open(os.path.join(data_folder, "lexicon_test.csv"), "w") as f:
        f.writelines(test_lines)


@dataclass
class LSRow:
    snt_id: str
    spk_id: str
    duration: float
    file_path: str
    words: str


def process_line(wav_file, text_dict) -> LSRow:
    snt_id = wav_file.split("/")[-1].replace(".flac", "")
    spk_id = "-".join(snt_id.split("-")[0:2])
    wrds = text_dict[snt_id]
    wrds = " ".join(wrds.split("_"))

    info = read_audio_info(wav_file)
    duration = info.num_frames / info.sample_rate

    return LSRow(
        snt_id=snt_id,
        spk_id=spk_id,
        duration=duration,
        file_path=wav_file,
        words=wrds,
    )


def create_csv(save_folder, wav_lst, text_dict, split, select_n_sentences):
    """
    Create the dataset csv file given a list of wav files.

    Arguments
    ---------
    save_folder : str
        Location of the folder for storing the csv.
    wav_lst : list
        The list of wav files of a given data split.
    text_dict : list
        The dictionary containing the text of each sentence.
    split : str
        The name of the current data split.
    select_n_sentences : int, optional
        The number of sentences to select.

    Returns
    -------
    None
    """
    # Setting path for the csv file
    csv_file = os.path.join(save_folder, split + ".csv")
    if os.path.exists(csv_file):
        logger.info("Csv file %s already exists, not recreating." % csv_file)
        return

    # Preliminary prints
    msg = "Creating csv lists in  %s..." % (csv_file)
    logger.info(msg)

    csv_lines = [["ID", "duration", "wav", "spk_id", "wrd"]]

    snt_cnt = 0
    line_processor = functools.partial(process_line, text_dict=text_dict)
    # Processing all the wav files in wav_lst
    # FLAC metadata reading is already fast, so we set a high chunk size
    # to limit main thread CPU bottlenecks
    for row in parallel_map(line_processor, wav_lst, chunk_size=8192):
        csv_line = [
            row.snt_id,
            str(row.duration),
            row.file_path,
            row.spk_id,
            row.words,
        ]

        # Appending current file to the csv_lines list
        csv_lines.append(csv_line)

        snt_cnt = snt_cnt + 1

        # parallel_map guarantees element ordering so we're OK
        if snt_cnt == select_n_sentences:
            break

    # Writing the csv_lines
    with open(csv_file, mode="w") as csv_f:
        csv_writer = csv.writer(
            csv_f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL
        )

        for line in csv_lines:
            csv_writer.writerow(line)

    # Final print
    msg = "%s successfully created!" % (csv_file)
    logger.info(msg)


def skip(splits, save_folder, conf):
    """
    Detect when the librispeech data prep can be skipped.

    Arguments
    ---------
    splits : list
        A list of the splits expected in the preparation.
    save_folder : str
        The location of the save directory
    conf : dict
        The configuration options to ensure they haven't changed.

    Returns
    -------
    bool
        if True, the preparation phase can be skipped.
        if False, it must be done.
    """

    # Checking csv files
    skip = True

    for split in splits:
        if not os.path.isfile(os.path.join(save_folder, split + ".csv")):
            skip = False

    #  Checking saved options
    save_opt = os.path.join(save_folder, OPT_FILE)
    if skip is True:
        if os.path.isfile(save_opt):
            opts_old = load_pkl(save_opt)
            if opts_old == conf:
                skip = True
            else:
                skip = False
        else:
            skip = False

    return skip


def text_to_dict(text_lst):
    """
    This converts lines of text into a dictionary-

    Arguments
    ---------
    text_lst : str
        Path to the file containing the librispeech text transcription.

    Returns
    -------
    dict
        The dictionary containing the text transcriptions for each sentence.

    """
    # Initialization of the text dictionary
    text_dict = {}
    # Reading all the transcription files is text_lst
    for file in text_lst:
        with open(file, "r") as f:
            # Reading all line of the transcription file
            for line in f:
                line_lst = line.strip().split(" ")
                text_dict[line_lst[0]] = "_".join(line_lst[1:])
    return text_dict


def check_librispeech_folders(data_folder, splits):
    """
    Check if the data folder actually contains the LibriSpeech dataset.

    If it does not, an error is raised.

    Arguments
    ---------
    data_folder : str
        The path to the directory with the data.
    splits : list
        The portions of the data to check.

    Raises
    ------
    OSError
        If LibriSpeech is not found at the specified path.
    """
    # Checking if all the splits exist
    for split in splits:
        split_folder = os.path.join(data_folder, split)
        if not os.path.exists(split_folder):
            err_msg = (
                "the folder %s does not exist (it is expected in the "
                "Librispeech dataset)" % split_folder
            )
            raise OSError(err_msg)


def download_librispeech_vocab_text(destination):
    """Download librispeech vocab file and unpack it.

    Arguments
    ---------
    destination : str
        Place to put vocab file.
    """
    f = "librispeech-vocab.txt"
    download_file(OPEN_SLR_11_LINK + f, destination)


def download_openslr_librispeech_lm(destination, rescoring_lm=True):
    """Download openslr librispeech lm and unpack it.

    Arguments
    ---------
    destination : str
        Place to put lm.
    rescoring_lm : bool
        Also download bigger 4grams model
    """
    os.makedirs(destination, exist_ok=True)
    for f in OPEN_SLR_11_NGRAM_MODELs:
        if f.startswith("4") and not rescoring_lm:
            continue
        d = os.path.join(destination, f)
        download_file(OPEN_SLR_11_LINK + f, d, unpack=True)


def download_sb_librispeech_lm(destination, rescoring_lm=True):
    """Download sb librispeech lm and unpack it.

    Arguments
    ---------
    destination : str
        Place to put lm.
    rescoring_lm : bool
        Also download bigger 4grams model
    """
    os.makedirs(destination, exist_ok=True)
    download_file(
        "https://www.dropbox.com/scl/fi/3fkkdlliavhveb5n3nsow/3gram_lm.arpa?rlkey=jgdrluppfut1pjminf3l3y106&dl=1",
        os.path.join(destination, "3-gram_sb.arpa"),
    )
    if rescoring_lm:
        download_file(
            "https://www.dropbox.com/scl/fi/roz46ee0ah2lvy5csno4z/4gram_lm.arpa?rlkey=2wt8ozb1mqgde9h9n9rp2yppz&dl=1",
            os.path.join(destination, "4-gram_sb.arpa"),
        )


Writing librispeech_prepare.py


In [ ]:
!mkdir hparams
%cd /content/speechbrain/recipes/LibriSpeech/ASR/mamba/hparams

mkdir: cannot create directory ‘hparams’: File exists
/content/speechbrain/recipes/LibriSpeech/ASR/mamba/hparams


In [ ]:
%%file mamba_BPE_1000.yaml
# ############################################################################
# Model: E2E ASR with Mamba- based  ASR
# Encoder: Mamba Encoder
# Decoder: Mamba Decoder
# Tokens: BPE with unigram
# losses: CTC+ NLL
# Training: Librispeech 100h
# Authors:  William Chittavong
# ############################################################################

# Seed needs to be set at top of yaml, before objects with parameters
seed: 2602
__set_seed: !apply:torch.manual_seed [!ref <seed>]
output_folder: !ref results/Mamba_100h_LM/<seed>
output_wer_folder: !ref <output_folder>/
save_folder: !ref <output_folder>/save
train_log: !ref <output_folder>/train_log.txt

# Language model (LM) pretraining
# NB: To avoid mismatch, the speech recognizer must be trained with the same
# tokenizer used for LM training. Here, we download everything from the
# speechbrain HuggingFace repository. However, a local path pointing to a
# directory containing the lm.ckpt and tokenizer.ckpt may also be specified
# instead. E.g if you want to use your own LM / tokenizer.
pretrained_lm_tokenizer_path: speechbrain/asr-crdnn-rnnlm-librispeech

# Data files
data_folder: /content/speechbrain/LibriSpeech # e,g./path/to/LibriSpeech

train_splits: ["train-clean-100"]
dev_splits: ["dev-clean"]
test_splits: ["test-clean"]
skip_prep: False
ckpt_interval_minutes: 15 # save checkpoint every N min
train_csv: !ref <save_folder>/train.csv
valid_csv: !ref <save_folder>/dev-clean.csv
test_csv:
   - !ref <save_folder>/test-clean.csv


# Data for augmentation
data_folder_noise: !ref <data_folder>/noise # The noisy sequences for data augmentation will automatically be downloaded here.
NOISE_DATASET_URL: https://www.dropbox.com/scl/fi/a09pj97s5ifan81dqhi4n/noises.zip?rlkey=j8b0n9kdjdr32o1f06t0cw5b7&dl=1
noise_annotation: !ref <save_folder>/noise.csv #The data manifest files are created by the data preparation script

####################### Training Parameters ####################################

number_of_epochs: 15
number_of_ctc_epochs: 5
batch_size: 8
lr: 1.0
ctc_weight: 0.5
sorting: ascending
dynamic_batching: False
precision: fp32 # bf16, fp16 or fp32

# dynamic batching parameters, if used
feats_hop_size: 0.01
max_batch_length: 20000 # in terms of frames
shuffle: True
batch_ordering: random
num_buckets: 20
dynamic_batch_sampler:
   max_batch_length: !ref <max_batch_length>
   shuffle: !ref <shuffle>
   batch_ordering: !ref <batch_ordering>
   num_buckets: !ref <num_buckets>

# Feature parameters
sample_rate: 16000
n_fft: 400
n_mels: 40

opt_class: !name:torch.optim.Adadelta
   lr: !ref <lr>
   rho: 0.95
   eps: 1.e-8

# Dataloader options
num_workers: 4
train_dataloader_opts:
   num_workers: !ref <num_workers>
   batch_size: !ref <batch_size>

valid_dataloader_opts:
   num_workers: !ref <num_workers>
   batch_size: !ref <batch_size>

test_dataloader_opts:
   batch_size: 1

####################### Model Parameters #######################################

dropout: 0.15
dnn_blocks: 2
dnn_neurons: 512
emb_size: 128
dec_neurons: 1024
blank_index: 0
bos_index: 0
eos_index: 0

output_neurons: 1000  # Number of tokens (same as LM)




hidden_size: 1024
state_size: 16
num_hidden_layers: 4
layer_norm_epsilon: 1e-05
pad_token_id: 0

expand: 2
conv_kernel: 4
use_bias: False
use_conv_bias: True
hidden_act: 'silu'
initializer_range: 0.1
residual_in_fp32: True
time_step_rank: 'auto'
time_step_scale: 1.0
time_step_min: 0.001
time_step_max: 0.1
time_step_init_scheme: 'random'
time_step_floor: 0.0001
rescale_prenorm_residual: False
use_cache: True

mamba_dec: !new:speechbrain.lobes.models.mamba.MambaDecoder
  vocab_size: !ref <output_neurons>
  hidden_size: !ref <hidden_size>
  state_size: !ref <state_size>
  num_hidden_layers: !ref <num_hidden_layers>
  layer_norm_epsilon: !ref <layer_norm_epsilon>
  pad_token_id: !ref <pad_token_id>
  bos_token_id: !ref <bos_index>
  eos_token_id: !ref <eos_index>
  expand: !ref <expand>
  conv_kernel: !ref <conv_kernel>
  use_bias: !ref <use_bias>
  use_conv_bias: !ref <use_conv_bias>
  hidden_act: !ref <hidden_act>
  initializer_range: !ref <initializer_range>
  residual_in_fp32: !ref <residual_in_fp32>
  time_step_rank: !ref <time_step_rank>
  time_step_scale: !ref <time_step_scale>
  time_step_min: !ref <time_step_min>
  time_step_max: !ref <time_step_max>
  time_step_init_scheme: !ref <time_step_init_scheme>
  time_step_floor: !ref <time_step_floor>
  rescale_prenorm_residual: !ref <rescale_prenorm_residual>
  use_cache: !ref <use_cache>


# Decoding parameters
min_decode_ratio: 0.0
max_decode_ratio: 1.0
valid_beam_size: 80
test_beam_size: 80
eos_threshold: 1.5
using_max_attn_shift: True
max_attn_shift: 240
temperature: 1.25
temperature_lm: 1.25

# Scoring parameters
lm_weight: 0.5
coverage_penalty: 1.5

epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
   limit: !ref <number_of_epochs>

normalize: !new:speechbrain.processing.features.InputNormalization
   norm_type: global

compute_features: !new:speechbrain.lobes.features.Fbank
   sample_rate: !ref <sample_rate>
   n_fft: !ref <n_fft>
   n_mels: !ref <n_mels>


enc: !new:speechbrain.lobes.models.mamba.MambaEncoder
    vocab_size: !ref <output_neurons>
    hidden_size: !ref <hidden_size>
    state_size: !ref <state_size>
    num_hidden_layers: !ref <num_hidden_layers>
    layer_norm_epsilon: !ref <layer_norm_epsilon>
    pad_token_id: !ref <pad_token_id>
    bos_token_id: !ref <bos_index>
    eos_token_id: !ref <eos_index>
    expand: !ref <expand>
    conv_kernel: !ref <conv_kernel>
    use_bias: !ref <use_bias>
    use_conv_bias: !ref <use_conv_bias>
    hidden_act: !ref <hidden_act>
    initializer_range: !ref <initializer_range>
    residual_in_fp32: !ref <residual_in_fp32>
    time_step_rank: !ref <time_step_rank>
    time_step_scale: !ref <time_step_scale>
    time_step_min: !ref <time_step_min>
    time_step_max: !ref <time_step_max>
    time_step_init_scheme: !ref <time_step_init_scheme>
    time_step_floor: !ref <time_step_floor>
    rescale_prenorm_residual: !ref <rescale_prenorm_residual>
    use_cache: !ref <use_cache>

emb: !new:speechbrain.nnet.embedding.Embedding
   num_embeddings: !ref <output_neurons>
   embedding_dim: !ref <emb_size>

dec: !new:speechbrain.nnet.RNN.AttentionalRNNDecoder
   enc_dim: !ref <dnn_neurons>
   input_size: !ref <emb_size>
   rnn_type: gru
   attn_type: location
   hidden_size: !ref <dec_neurons>
   attn_dim: 1024
   num_layers: 1
   scaling: 1.0
   channels: 10
   kernel_size: 100
   re_init: True
   dropout: !ref <dropout>


feats_to_mamba_lin: !new:speechbrain.nnet.linear.Linear
   input_size: !ref <n_mels>
   n_neurons : !ref <hidden_size>

emb_to_mamba_lin: !new:speechbrain.nnet.linear.Linear
   input_size: !ref <emb_size>
   n_neurons: !ref <hidden_size>
ctc_lin: !new:speechbrain.nnet.linear.Linear
   input_size: !ref <hidden_size>
   n_neurons: !ref <output_neurons>

seq_lin: !new:speechbrain.nnet.linear.Linear
   input_size: !ref <hidden_size>
   n_neurons: !ref <output_neurons>

log_softmax: !new:speechbrain.nnet.activations.Softmax
   apply_log: True

ctc_cost: !name:speechbrain.nnet.losses.ctc_loss
   blank_index: !ref <blank_index>

seq_cost: !name:speechbrain.nnet.losses.nll_loss
   label_smoothing: 0.1

# This is the RNNLM that is used according to the Huggingface repository
# NB: It has to match the pre-trained RNNLM!!
lm_model: !new:speechbrain.lobes.models.RNNLM.RNNLM
   output_neurons: !ref <output_neurons>
   embedding_dim: !ref <emb_size>
   activation: !name:torch.nn.LeakyReLU
   dropout: 0.0
   rnn_layers: 2
   rnn_neurons: 2048
   dnn_blocks: 1
   dnn_neurons: 512
   return_hidden: True  # For inference

tokenizer: !new:sentencepiece.SentencePieceProcessor

# Models
modules:
   enc: !ref <enc>
   emb: !ref <emb>
   mamba_dec: !ref <mamba_dec>
   dec: !ref <dec>
   ctc_lin: !ref <ctc_lin>
   seq_lin: !ref <seq_lin>
   feats_to_mamba_lin: !ref <feats_to_mamba_lin>
   emb_to_mamba_lin: !ref <emb_to_mamba_lin>
   normalize: !ref <normalize>
   lm_model: !ref <lm_model>

model: !new:torch.nn.ModuleList
   - [!ref <enc>, !ref <emb>,!ref <mamba_dec>,!ref <dec>, !ref <ctc_lin>, !ref <seq_lin>]

############################## Decoding & optimiser ############################

coverage_scorer: !new:speechbrain.decoders.scorer.CoverageScorer
   vocab_size: !ref <output_neurons>

rnnlm_scorer: !new:speechbrain.decoders.scorer.RNNLMScorer
   language_model: !ref <lm_model>
   temperature: !ref <temperature_lm>

scorer: !new:speechbrain.decoders.scorer.ScorerBuilder
   full_scorers: [!ref <rnnlm_scorer>,
                  !ref <coverage_scorer>]
   weights:
      rnnlm: !ref <lm_weight>
      coverage: !ref <coverage_penalty>

# Search
valid_search: !new:speechbrain.decoders.S2SRNNBeamSearcher
   embedding: !ref <emb>
   decoder: !ref <dec>
   linear: !ref <seq_lin>
   bos_index: !ref <bos_index>
   eos_index: !ref <eos_index>
   min_decode_ratio: !ref <min_decode_ratio>
   max_decode_ratio: !ref <max_decode_ratio>
   beam_size: !ref <valid_beam_size>
   eos_threshold: !ref <eos_threshold>
   using_max_attn_shift: !ref <using_max_attn_shift>
   max_attn_shift: !ref <max_attn_shift>
   temperature: !ref <temperature>

test_search: !new:speechbrain.decoders.S2SRNNBeamSearcher
   embedding: !ref <emb>
   decoder: !ref <dec>
   linear: !ref <seq_lin>
   bos_index: !ref <bos_index>
   eos_index: !ref <eos_index>
   min_decode_ratio: !ref <min_decode_ratio>
   max_decode_ratio: !ref <max_decode_ratio>
   beam_size: !ref <test_beam_size>
   eos_threshold: !ref <eos_threshold>
   using_max_attn_shift: !ref <using_max_attn_shift>
   max_attn_shift: !ref <max_attn_shift>
   temperature: !ref <temperature>
   scorer: !ref <scorer>

lr_annealing: !new:speechbrain.nnet.schedulers.NewBobScheduler
   initial_value: !ref <lr>
   improvement_threshold: 0.0025
   annealing_factor: 0.8
   patient: 0

############################## Augmentations ###################################

prepare_noise_data: !name:speechbrain.augment.preparation.prepare_dataset_from_URL
   URL: !ref <NOISE_DATASET_URL>
   dest_folder: !ref <data_folder_noise>
   ext: wav
   csv_file: !ref <noise_annotation>

# Add noise to input signal
add_noise: !new:speechbrain.augment.time_domain.AddNoise
   csv_file: !ref <noise_annotation>
   snr_low: 0
   snr_high: 15
   noise_sample_rate: !ref <sample_rate>
   clean_sample_rate: !ref <sample_rate>
   num_workers: !ref <num_workers>

# Speed perturbation
speed_perturb: !new:speechbrain.augment.time_domain.SpeedPerturb
   orig_freq: !ref <sample_rate>
   speeds: [95, 100, 105]

# Frequency drop: randomly drops a number of frequency bands to zero.
drop_freq: !new:speechbrain.augment.time_domain.DropFreq
   drop_freq_low: 0
   drop_freq_high: 1
   drop_freq_count_low: 1
   drop_freq_count_high: 3
   drop_freq_width: 0.05

# Time drop: randomly drops a number of temporal chunks.
drop_chunk: !new:speechbrain.augment.time_domain.DropChunk
   drop_length_low: 1000
   drop_length_high: 2000
   drop_count_low: 1
   drop_count_high: 5

# Augmenter: Combines previously defined augmentations to perform data augmentation
wav_augment: !new:speechbrain.augment.augmenter.Augmenter
   concat_original: True
   min_augmentations: 4
   max_augmentations: 4
   augment_prob: 1.0
   augmentations: [
      !ref <add_noise>,
      !ref <speed_perturb>,
      !ref <drop_freq>,
      !ref <drop_chunk>]

############################## Logging and Pretrainer ##########################

checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
   checkpoints_dir: !ref <save_folder>
   recoverables:
      model: !ref <model>
      scheduler: !ref <lr_annealing>
      normalizer: !ref <normalize>
      counter: !ref <epoch_counter>

train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
   save_file: !ref <train_log>

error_rate_computer: !name:speechbrain.utils.metric_stats.ErrorRateStats

cer_computer: !name:speechbrain.utils.metric_stats.ErrorRateStats
   split_tokens: True

# The pretrainer allows a mapping between pretrained files and instances that
# are declared in the yaml. E.g here, we will download the file lm.ckpt
# and it will be loaded into "lm" which is pointing to the <lm_model> defined
# before.
pretrainer: !new:speechbrain.utils.parameter_transfer.Pretrainer
   collect_in: !ref <save_folder>
   loadables:
      lm: !ref <lm_model>
      tokenizer: !ref <tokenizer>
   paths:
      lm: !ref <pretrained_lm_tokenizer_path>/lm.ckpt
      tokenizer: !ref <pretrained_lm_tokenizer_path>/tokenizer.ckpt


Overwriting mamba_BPE_1000.yaml


In [ ]:
%cd /content/speechbrain/recipes/LibriSpeech/ASR/mamba

/content/speechbrain/recipes/LibriSpeech/ASR/mamba


In [ ]:
%%file train.py
#!/usr/bin/env/python3
"""Recipe for training a sequence-to-sequence ASR system with librispeech.
The system employs an encoder, a decoder, and an attention mechanism
between them. Decoding is performed with beamsearch coupled with a neural
language model.
To run this recipe, do the following:
> python train.py hparams/train_BPE1000.yaml
With the default hyperparameters, the system employs a CRDNN encoder.
The decoder is based on a standard  GRU. Beamsearch coupled with a RNN
language model is used  on the top of decoder probabilities.
The neural network is trained on both CTC and negative-log likelihood
targets and sub-word units estimated with Byte Pairwise Encoding (BPE)
are used as basic recognition tokens. Training is performed on the full
LibriSpeech dataset (960 h).
The experiment file is flexible enough to support a large variety of
different systems. By properly changing the parameter files, you can try
different encoders, decoders, tokens (e.g, characters instead of BPE),
training split (e.g, train-clean 100 rather than the full one), and many
other possible variations.
This recipe assumes that the tokenizer and the LM are already trained.
To avoid token mismatches, the tokenizer used for the acoustic model is
the same use for the LM.  The recipe downloads the pre-trained tokenizer
and LM.
If you would like to train a full system from scratch do the following:
1- Train a tokenizer (see ../../Tokenizer)
2- Train a language model (see ../../LM)
3- Train the acoustic model (with this code).
Authors
 * Ju-Chieh Chou 2020
 * Mirco Ravanelli 2020
 * Abdel Heba 2020
 * Peter Plantinga 2020
 * Samuele Cornell 2020
 * Andreas Nautsch 2021
"""

import sys
import torch
import logging
import speechbrain as sb
from speechbrain.utils.distributed import run_on_main, if_main_process
from hyperpyyaml import load_hyperpyyaml
from pathlib import Path

logger = logging.getLogger(__name__)


# Define training procedure
class ASR(sb.Brain):
    def compute_forward(self, batch, stage):
        """Forward computations from the waveform batches to the output probabilities."""
        batch = batch.to(self.device)
        wavs, wav_lens = batch.sig
        tokens_bos, _ = batch.tokens_bos
        wavs, wav_lens = wavs.to(self.device), wav_lens.to(self.device)

        # Add waveform augmentation if specified.
        if stage == sb.Stage.TRAIN and hasattr(self.hparams, "wav_augment"):
            wavs, wav_lens = self.hparams.wav_augment(wavs, wav_lens)
            tokens_bos = self.hparams.wav_augment.replicate_labels(tokens_bos)

        # Forward pass
        feats = self.hparams.compute_features(wavs)
        feats = self.modules.normalize(feats, wav_lens)

        feats = self.modules.feats_to_mamba_lin(feats)
        print("feats after linear",feats.shape)
        x = self.modules.enc(feats.detach())
        print("\n encoded states",x.shape)
        e_in = self.modules.emb(tokens_bos)  # y_in bos + tokens
        print("\n emb tokens ",e_in.shape)
        src_mask = (torch.arange(x.size(1), device=self.device).unsqueeze(0) < wav_lens.unsqueeze(1)).unsqueeze(1).unsqueeze(1)
        print("\n src mask", src_mask.shape)
        tgt_mask = (torch.triu(torch.ones(e_in.size(1), e_in.size(1), device=self.device), diagonal=1) == 0).unsqueeze(0).unsqueeze(0)
        print("\n tgt mask", tgt_mask.shape)
        e_in = self.modules.emb_to_mamba_lin(e_in)
        h = self.modules.mamba_dec(embeds = e_in,encoder_output =  x, src_mask=src_mask, tgt_mask=tgt_mask)

        # Output layer for seq2seq log-probabilities
        logits = self.modules.seq_lin(h)
        p_seq = self.hparams.log_softmax(logits)

        # Compute outputs
        p_ctc, p_tokens = None, None
        if stage == sb.Stage.TRAIN:
            current_epoch = self.hparams.epoch_counter.current
            if current_epoch <= self.hparams.number_of_ctc_epochs:
                # Output layer for ctc log-probabilities
                logits = self.modules.ctc_lin(x)
                p_ctc = self.hparams.log_softmax(logits)
        else:
            if stage == sb.Stage.VALID:
                # Get token strings from index prediction
                p_tokens, _, _, _ = self.hparams.valid_search(x, wav_lens)
            else:
                p_tokens, _, _, _ = self.hparams.test_search(x, wav_lens)

        return p_ctc, p_seq, wav_lens, p_tokens

    def compute_objectives(self, predictions, batch, stage):
        """Computes the loss (CTC+NLL) given predictions and targets."""

        current_epoch = self.hparams.epoch_counter.current
        p_ctc, p_seq, wav_lens, predicted_tokens = predictions

        ids = batch.id
        tokens_eos, tokens_eos_lens = batch.tokens_eos
        tokens, tokens_lens = batch.tokens

        # Labels must be extended if parallel augmentation or concatenated
        # augmentation was performed on the input (increasing the time dimension)
        if stage == sb.Stage.TRAIN and hasattr(self.hparams, "wav_augment"):
            (
                tokens,
                tokens_lens,
                tokens_eos,
                tokens_eos_lens,
            ) = self.hparams.wav_augment.replicate_multiple_labels(
                tokens, tokens_lens, tokens_eos, tokens_eos_lens
            )

        loss_seq = self.hparams.seq_cost(
            p_seq, tokens_eos, length=tokens_eos_lens
        )

        # Add ctc loss if necessary
        if (
            stage == sb.Stage.TRAIN
            and current_epoch <= self.hparams.number_of_ctc_epochs
        ):
            loss_ctc = self.hparams.ctc_cost(
                p_ctc, tokens, wav_lens, tokens_lens
            )
            loss = self.hparams.ctc_weight * loss_ctc
            loss += (1 - self.hparams.ctc_weight) * loss_seq
        else:
            loss = loss_seq

        if stage != sb.Stage.TRAIN:
            # Decode token terms to words
            predicted_words = [
                self.tokenizer.decode_ids(utt_seq).split(" ")
                for utt_seq in predicted_tokens
            ]
            target_words = [wrd.split(" ") for wrd in batch.wrd]
            self.wer_metric.append(ids, predicted_words, target_words)
            self.cer_metric.append(ids, predicted_words, target_words)

        return loss

    def on_stage_start(self, stage, epoch):
        """Gets called at the beginning of each epoch"""
        if stage != sb.Stage.TRAIN:
            self.cer_metric = self.hparams.cer_computer()
            self.wer_metric = self.hparams.error_rate_computer()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        # Compute/store important stats
        stage_stats = {"loss": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats
        else:
            stage_stats["CER"] = self.cer_metric.summarize("error_rate")
            stage_stats["WER"] = self.wer_metric.summarize("error_rate")

        # Perform end-of-iteration things, like annealing, logging, etc.
        if stage == sb.Stage.VALID:
            old_lr, new_lr = self.hparams.lr_annealing(stage_stats["WER"])
            sb.nnet.schedulers.update_learning_rate(self.optimizer, new_lr)
            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": old_lr},
                train_stats=self.train_stats,
                valid_stats=stage_stats,
            )
            self.checkpointer.save_and_keep_only(
                meta={"WER": stage_stats["WER"]},
                min_keys=["WER"],
            )
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stage_stats,
            )
            if if_main_process():
                with open(self.hparams.test_wer_file, "w") as w:
                    self.wer_metric.write_stats(w)


def dataio_prepare(hparams):
    """This function prepares the datasets to be used in the brain class.
    It also defines the data processing pipeline through user-defined functions.
    """
    data_folder = hparams["data_folder"]

    train_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["train_csv"],
        replacements={"data_root": data_folder},
    )

    if hparams["sorting"] == "ascending":
        # we sort training data to speed up training and get better results.
        train_data = train_data.filtered_sorted(sort_key="duration")
        # when sorting do not shuffle in dataloader ! otherwise is pointless
        hparams["train_dataloader_opts"]["shuffle"] = False

    elif hparams["sorting"] == "descending":
        train_data = train_data.filtered_sorted(
            sort_key="duration", reverse=True
        )
        # when sorting do not shuffle in dataloader ! otherwise is pointless
        hparams["train_dataloader_opts"]["shuffle"] = False

    elif hparams["sorting"] == "random":
        pass

    else:
        raise NotImplementedError(
            "sorting must be random, ascending or descending"
        )

    valid_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["valid_csv"],
        replacements={"data_root": data_folder},
    )
    valid_data = valid_data.filtered_sorted(sort_key="duration")

    # test is separate
    test_datasets = {}
    for csv_file in hparams["test_csv"]:
        name = Path(csv_file).stem
        test_datasets[name] = sb.dataio.dataset.DynamicItemDataset.from_csv(
            csv_path=csv_file, replacements={"data_root": data_folder}
        )
        test_datasets[name] = test_datasets[name].filtered_sorted(
            sort_key="duration"
        )

    datasets = [train_data, valid_data] + [i for k, i in test_datasets.items()]

    # We get the tokenizer as we need it to encode the labels when creating
    # mini-batches.
    tokenizer = hparams["tokenizer"]

    # 2. Define audio pipeline:
    @sb.utils.data_pipeline.takes("wav")
    @sb.utils.data_pipeline.provides("sig")
    def audio_pipeline(wav):
        sig = sb.dataio.dataio.read_audio(wav)
        return sig

    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline)

    # 3. Define text pipeline:
    @sb.utils.data_pipeline.takes("wrd")
    @sb.utils.data_pipeline.provides(
        "wrd", "tokens_list", "tokens_bos", "tokens_eos", "tokens"
    )
    def text_pipeline(wrd):
        yield wrd
        tokens_list = tokenizer.encode_as_ids(wrd)
        yield tokens_list
        tokens_bos = torch.LongTensor([hparams["bos_index"]] + (tokens_list))
        yield tokens_bos
        tokens_eos = torch.LongTensor(tokens_list + [hparams["eos_index"]])
        yield tokens_eos
        tokens = torch.LongTensor(tokens_list)
        yield tokens

    sb.dataio.dataset.add_dynamic_item(datasets, text_pipeline)

    # 4. Set output:
    sb.dataio.dataset.set_output_keys(
        datasets,
        ["id", "sig", "wrd", "tokens_bos", "tokens_eos", "tokens"],
    )
    train_batch_sampler = None
    valid_batch_sampler = None
    if hparams["dynamic_batching"]:
        from speechbrain.dataio.sampler import DynamicBatchSampler  # noqa
        from speechbrain.dataio.dataloader import SaveableDataLoader  # noqa
        from speechbrain.dataio.batch import PaddedBatch  # noqa

        dynamic_hparams = hparams["dynamic_batch_sampler"]
        hop_size = hparams["feats_hop_size"]

        train_batch_sampler = DynamicBatchSampler(
            train_data,
            length_func=lambda x: x["duration"] * (1 / hop_size),
            **dynamic_hparams,
        )

        valid_batch_sampler = DynamicBatchSampler(
            valid_data,
            length_func=lambda x: x["duration"] * (1 / hop_size),
            **dynamic_hparams,
        )

    return (
        train_data,
        valid_data,
        test_datasets,
        train_batch_sampler,
        valid_batch_sampler,
    )


if __name__ == "__main__":
    # CLI:
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])

    # create ddp_group with the right communication protocol
    sb.utils.distributed.ddp_init_group(run_opts)

    with open(hparams_file) as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # Dataset prep (parsing Librispeech)
    from librispeech_prepare import prepare_librispeech  # noqa

    # multi-gpu (ddp) save data preparation
    run_on_main(
        prepare_librispeech,
        kwargs={
            "data_folder": hparams["data_folder"],
            "tr_splits": hparams["train_splits"],
            "dev_splits": hparams["dev_splits"],
            "te_splits": hparams["test_splits"],
            "save_folder": hparams["save_folder"],
            "merge_lst": hparams["train_splits"],
            "merge_name": "train.csv",
            "skip_prep": hparams["skip_prep"],
        },
    )
    run_on_main(hparams["prepare_noise_data"])

    # here we create the datasets objects as well as tokenization and encoding
    (
        train_data,
        valid_data,
        test_datasets,
        train_bsampler,
        valid_bsampler,
    ) = dataio_prepare(hparams)

    # We download the pretrained LM from HuggingFace (or elsewhere depending on
    # the path given in the YAML file). The tokenizer is loaded at the same time.
    run_on_main(hparams["pretrainer"].collect_files)
    hparams["pretrainer"].load_collected()

    # Trainer initialization
    asr_brain = ASR(
        modules=hparams["modules"],
        opt_class=hparams["opt_class"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )

    # We dynamically add the tokenizer to our brain class.
    # NB: This tokenizer corresponds to the one used for the LM!!
    asr_brain.tokenizer = hparams["tokenizer"]
    train_dataloader_opts = hparams["train_dataloader_opts"]
    valid_dataloader_opts = hparams["valid_dataloader_opts"]

    if train_bsampler is not None:
        train_dataloader_opts = {"batch_sampler": train_bsampler}
    if valid_bsampler is not None:
        valid_dataloader_opts = {"batch_sampler": valid_bsampler}

    # Training
    asr_brain.fit(
        asr_brain.hparams.epoch_counter,
        train_data,
        valid_data,
        train_loader_kwargs=train_dataloader_opts,
        valid_loader_kwargs=valid_dataloader_opts,
    )

    import os

    # Testing
    if not os.path.exists(hparams["output_wer_folder"]):
        os.makedirs(hparams["output_wer_folder"])

    for k in test_datasets.keys():  # keys are test_clean, test_other etc
        asr_brain.hparams.test_wer_file = os.path.join(
            hparams["output_wer_folder"], f"wer_{k}.txt"
        )
        asr_brain.evaluate(
            test_datasets[k],
            test_loader_kwargs=hparams["test_dataloader_opts"],
            min_key="WER",
        )


Overwriting train.py


In [ ]:
!rm -rf /content/speechbrain/recipes/LibriSpeech/ASR/mamba/results/Mamba_100h_LM

In [ ]:
!python train.py hparams/mamba_BPE_1000.yaml

/usr/lib/python3.10/inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/Mamba_100h_LM/2602
librispeech_prepare - Data_preparation...
librispeech_prepare - Creating csv lists in  results/Mamba_100h_LM/2602/save/train-clean-100.csv...
  0% 0/28539 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100% 28539/28539 [02:00<00:00, 235.95it/s]
librispeech_prepare - results/Mamba_100h_LM/2602/save/train-clean-100.csv successfully created!
librispeech_prepare - Crea